# Risk Analysis Notebook
## Monte Carlo Simulation, VaR, and CVaR Analysis

This notebook provides comprehensive risk analysis tools for financial and economic data:
- **Value at Risk (VaR)**: Maximum expected loss at a given confidence level
- **Conditional Value at Risk (CVaR)**: Expected loss beyond VaR threshold
- **Monte Carlo Simulation**: Probabilistic scenario generation
- **Stress Testing**: Impact analysis under extreme conditions

### Key Features:
- Multiple VaR calculation methods (Historical, Parametric, Monte Carlo)
- Advanced Monte Carlo simulations with various distributions
- Portfolio risk metrics and optimization
- Stress testing scenarios and backtesting

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import scipy.stats as stats
from scipy.optimize import minimize
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

print("📚 Libraries imported successfully")
print("🎯 Random seed set for reproducible results")

In [ ]:
# Import custom risk models
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '..', 'src'))

try:
    from models.risk import VaRCalculator, MonteCarloSimulator, StressTesting
    from utils.plotting import ECONET_COLORS
    print("✅ Custom risk models imported successfully")
except ImportError as e:
    print(f"⚠️ Import warning: {e}")
    print("📝 Using fallback implementations")

## 1. Data Generation and Preparation

Generate synthetic financial returns data with realistic characteristics:
- Fat tails (higher probability of extreme events)
- Volatility clustering
- Autocorrelation patterns

In [ ]:
# Parameters (can be overridden by Streamlit)
var_confidence = 0.95
n_simulations = 1000

def generate_financial_returns(n_periods=252, assets=['Stock', 'Bond', 'Commodity']):
    """
    Generate synthetic financial returns with realistic properties
    """
    dates = pd.date_range('2020-01-01', periods=n_periods, freq='D')
    
    returns_data = {}
    
    for i, asset in enumerate(assets):
        # Base parameters
        mu = [0.0005, 0.0002, 0.0003][i]  # Daily expected returns
        base_vol = [0.02, 0.008, 0.025][i]  # Base volatility
        
        # Generate returns with volatility clustering (GARCH-like)
        returns = []
        vol = base_vol
        
        for t in range(n_periods):
            # Volatility update (simplified GARCH)
            if t > 0:
                vol = 0.8 * vol + 0.1 * base_vol + 0.1 * abs(returns[-1])
            
            # Generate return with fat tails (t-distribution)
            if np.random.random() < 0.05:  # 5% chance of extreme event
                ret = stats.t.rvs(df=3, scale=vol * 2) + mu
            else:
                ret = np.random.normal(mu, vol)
            
            returns.append(ret)
        
        returns_data[asset] = returns
    
    # Add some correlation
    corr_matrix = np.array([
        [1.0, 0.3, 0.5],
        [0.3, 1.0, 0.2],
        [0.5, 0.2, 1.0]
    ])
    
    # Apply correlation (simplified)
    returns_matrix = np.array(list(returns_data.values())).T
    
    df = pd.DataFrame(returns_matrix, columns=assets, index=dates)
    
    return df

# Generate returns data
returns_data = generate_financial_returns()
print(f"📊 Generated {len(returns_data)} days of returns data")
print(f"💼 Assets: {list(returns_data.columns)}")
print(f"📈 Date range: {returns_data.index[0].strftime('%Y-%m-%d')} to {returns_data.index[-1].strftime('%Y-%m-%d')}")

# Display summary statistics
print("\n📊 Returns Summary Statistics:")
summary_stats = returns_data.describe()
print(summary_stats.round(6))

In [ ]:
# Visualize returns data
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=['Cumulative Returns', 'Daily Returns Distribution', 
                   'Rolling Volatility (30-day)', 'Returns Correlation'],
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"type": "heatmap"}]]
)

# 1. Cumulative returns
cumulative_returns = (1 + returns_data).cumprod()
for col in returns_data.columns:
    fig.add_trace(
        go.Scatter(x=returns_data.index, y=cumulative_returns[col], 
                  name=f'{col} Cumulative', mode='lines'),
        row=1, col=1
    )

# 2. Returns distribution (histogram)
for col in returns_data.columns:
    fig.add_trace(
        go.Histogram(x=returns_data[col], name=f'{col} Distribution', 
                    opacity=0.7, nbinsx=50),
        row=1, col=2
    )

# 3. Rolling volatility
rolling_vol = returns_data.rolling(30).std() * np.sqrt(252)  # Annualized
for col in returns_data.columns:
    fig.add_trace(
        go.Scatter(x=returns_data.index, y=rolling_vol[col], 
                  name=f'{col} Volatility', mode='lines'),
        row=2, col=1
    )

# 4. Correlation heatmap
corr_matrix = returns_data.corr()
fig.add_trace(
    go.Heatmap(z=corr_matrix.values, 
               x=corr_matrix.columns, 
               y=corr_matrix.columns,
               colorscale='RdBu',
               zmid=0,
               showscale=True),
    row=2, col=2
)

fig.update_layout(
    title='📈 Financial Returns Analysis',
    height=800,
    showlegend=True
)

fig.show()
print("📈 Returns data visualization complete")

## 2. Value at Risk (VaR) Analysis

Calculate VaR using multiple methodologies and compare results.

In [ ]:
# VaR Calculation Functions
class RiskCalculator:
    def __init__(self, confidence_level=0.95):
        self.confidence_level = confidence_level
        self.alpha = 1 - confidence_level
    
    def historical_var(self, returns):
        """Calculate VaR using historical method"""
        return np.percentile(returns, self.alpha * 100)
    
    def parametric_var(self, returns):
        """Calculate VaR using parametric method (normal distribution)"""
        mu = np.mean(returns)
        sigma = np.std(returns)
        return stats.norm.ppf(self.alpha, mu, sigma)
    
    def monte_carlo_var(self, returns, n_simulations=10000):
        """Calculate VaR using Monte Carlo simulation"""
        # Fit distribution to returns
        mu = np.mean(returns)
        sigma = np.std(returns)
        
        # Generate simulations
        simulated_returns = np.random.normal(mu, sigma, n_simulations)
        
        return np.percentile(simulated_returns, self.alpha * 100)
    
    def conditional_var(self, returns, var_value):
        """Calculate Conditional VaR (Expected Shortfall)"""
        tail_losses = returns[returns <= var_value]
        return np.mean(tail_losses) if len(tail_losses) > 0 else var_value
    
    def calculate_all_metrics(self, returns):
        """Calculate comprehensive risk metrics"""
        # VaR calculations
        hist_var = self.historical_var(returns)
        param_var = self.parametric_var(returns)
        mc_var = self.monte_carlo_var(returns, n_simulations)
        
        # CVaR calculations
        hist_cvar = self.conditional_var(returns, hist_var)
        param_cvar = self.conditional_var(returns, param_var)
        mc_cvar = self.conditional_var(returns, mc_var)
        
        # Additional metrics
        volatility = np.std(returns) * np.sqrt(252)  # Annualized
        skewness = stats.skew(returns)
        kurtosis = stats.kurtosis(returns)
        sharpe_ratio = np.mean(returns) / np.std(returns) * np.sqrt(252)
        
        return {
            'Historical_VaR': hist_var,
            'Parametric_VaR': param_var,
            'MonteCarlo_VaR': mc_var,
            'Historical_CVaR': hist_cvar,
            'Parametric_CVaR': param_cvar,
            'MonteCarlo_CVaR': mc_cvar,
            'Volatility': volatility,
            'Skewness': skewness,
            'Kurtosis': kurtosis,
            'Sharpe_Ratio': sharpe_ratio
        }

# Calculate risk metrics for each asset
risk_calculator = RiskCalculator(confidence_level=var_confidence)
risk_metrics = {}

print(f"⚡ Calculating risk metrics at {var_confidence*100}% confidence level...")

for asset in returns_data.columns:
    returns = returns_data[asset].dropna()
    metrics = risk_calculator.calculate_all_metrics(returns)
    risk_metrics[asset] = metrics
    
    print(f"\n📊 {asset} Risk Metrics:")
    print(f"  VaR (Historical): {metrics['Historical_VaR']:.4f}")
    print(f"  VaR (Parametric): {metrics['Parametric_VaR']:.4f}")
    print(f"  CVaR (Historical): {metrics['Historical_CVaR']:.4f}")
    print(f"  Volatility (Ann.): {metrics['Volatility']:.2f}%")
    print(f"  Sharpe Ratio: {metrics['Sharpe_Ratio']:.4f}")

print("\n✅ Risk metrics calculation complete")

In [ ]:
# Create comprehensive risk metrics DataFrame
risk_df = pd.DataFrame(risk_metrics).T

print("📊 COMPREHENSIVE RISK METRICS SUMMARY")
print("=" * 60)
print(risk_df.round(6))

# Identify highest and lowest risk assets
highest_risk = risk_df['Volatility'].idxmax()
lowest_risk = risk_df['Volatility'].idxmin()
best_sharpe = risk_df['Sharpe_Ratio'].idxmax()

print(f"\n🎯 RISK INSIGHTS:")
print(f"📈 Highest Risk Asset: {highest_risk} (Vol: {risk_df.loc[highest_risk, 'Volatility']:.2f}%)")
print(f"📉 Lowest Risk Asset: {lowest_risk} (Vol: {risk_df.loc[lowest_risk, 'Volatility']:.2f}%)")
print(f"🏆 Best Risk-Adjusted Return: {best_sharpe} (Sharpe: {risk_df.loc[best_sharpe, 'Sharpe_Ratio']:.4f})")

## 3. Monte Carlo Simulation

Perform Monte Carlo simulations to generate potential future scenarios.

In [ ]:
# Monte Carlo Simulation for Portfolio Scenarios
class MonteCarloRiskSimulator:
    def __init__(self, n_simulations=1000, time_horizon=30):
        self.n_simulations = n_simulations
        self.time_horizon = time_horizon
    
    def geometric_brownian_motion(self, S0, mu, sigma, corr_matrix=None):
        """
        Simulate asset prices using Geometric Brownian Motion
        """
        if isinstance(S0, (int, float)):
            # Single asset simulation
            dt = 1/252  # Daily steps
            
            # Generate random shocks
            Z = np.random.standard_normal((self.n_simulations, self.time_horizon))
            
            # Initialize price paths
            S = np.zeros((self.n_simulations, self.time_horizon + 1))
            S[:, 0] = S0
            
            # Generate paths
            for t in range(1, self.time_horizon + 1):
                S[:, t] = S[:, t-1] * np.exp((mu - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z[:, t-1])
            
            return S
        
        else:
            # Multi-asset simulation
            n_assets = len(S0)
            dt = 1/252
            
            # Generate correlated random shocks
            if corr_matrix is not None:
                L = np.linalg.cholesky(corr_matrix)
                Z = np.random.standard_normal((self.n_simulations, self.time_horizon, n_assets))
                Z = np.array([np.dot(z, L.T) for z in Z])
            else:
                Z = np.random.standard_normal((self.n_simulations, self.time_horizon, n_assets))
            
            # Initialize price paths
            S = np.zeros((self.n_simulations, self.time_horizon + 1, n_assets))
            S[:, 0, :] = S0
            
            # Generate paths
            for t in range(1, self.time_horizon + 1):
                for i in range(n_assets):
                    S[:, t, i] = S[:, t-1, i] * np.exp(
                        (mu[i] - 0.5 * sigma[i]**2) * dt + sigma[i] * np.sqrt(dt) * Z[:, t-1, i]
                    )
            
            return S
    
    def calculate_portfolio_scenarios(self, prices, weights):
        """
        Calculate portfolio value scenarios
        """
        if prices.ndim == 3:  # Multi-asset
            portfolio_values = np.sum(prices * weights, axis=2)
        else:  # Single asset
            portfolio_values = prices * weights
        
        return portfolio_values
    
    def calculate_scenario_statistics(self, scenarios):
        """
        Calculate statistics from simulation scenarios
        """
        final_values = scenarios[:, -1]
        
        stats = {
            'mean': np.mean(final_values),
            'median': np.median(final_values),
            'std': np.std(final_values),
            'min': np.min(final_values),
            'max': np.max(final_values),
            'var_95': np.percentile(final_values, 5),
            'var_99': np.percentile(final_values, 1),
            'cvar_95': np.mean(final_values[final_values <= np.percentile(final_values, 5)]),
            'cvar_99': np.mean(final_values[final_values <= np.percentile(final_values, 1)])
        }
        
        return stats

# Run Monte Carlo simulation
print(f"🎲 Running Monte Carlo simulation with {n_simulations:,} scenarios...")

# Simulation parameters
initial_portfolio_value = 100000  # $100K portfolio
time_horizon_days = 30  # 30 days

# Calculate expected returns and volatilities from historical data
expected_returns = returns_data.mean().values * 252  # Annualized
volatilities = returns_data.std().values * np.sqrt(252)  # Annualized
correlation_matrix = returns_data.corr().values

# Equal-weighted portfolio
n_assets = len(returns_data.columns)
weights = np.ones(n_assets) / n_assets
initial_prices = np.ones(n_assets) * (initial_portfolio_value / n_assets)

# Run simulation
simulator = MonteCarloRiskSimulator(n_simulations=n_simulations, time_horizon=time_horizon_days)
price_scenarios = simulator.geometric_brownian_motion(
    S0=initial_prices,
    mu=expected_returns,
    sigma=volatilities,
    corr_matrix=correlation_matrix
)

# Calculate portfolio scenarios
portfolio_scenarios = simulator.calculate_portfolio_scenarios(price_scenarios, weights)

# Calculate statistics
scenario_stats = simulator.calculate_scenario_statistics(portfolio_scenarios)

print("✅ Monte Carlo simulation complete")
print(f"📊 Simulation results for {time_horizon_days}-day horizon:")
print(f"  Expected Portfolio Value: ${scenario_stats['mean']:,.2f}")
print(f"  95% VaR: ${initial_portfolio_value - scenario_stats['var_95']:,.2f}")
print(f"  99% VaR: ${initial_portfolio_value - scenario_stats['var_99']:,.2f}")
print(f"  95% CVaR: ${initial_portfolio_value - scenario_stats['cvar_95']:,.2f}")
print(f"  Worst Case: ${scenario_stats['min']:,.2f}")
print(f"  Best Case: ${scenario_stats['max']:,.2f}")

In [ ]:
# Visualize Monte Carlo results
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=['Portfolio Value Scenarios', 'Final Value Distribution', 
                   'Risk Metrics Comparison', 'Percentile Analysis'],
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"type": "bar"}, {"secondary_y": False}]]
)

# 1. Portfolio scenarios (show subset for clarity)
days = np.arange(portfolio_scenarios.shape[1])
n_paths_to_show = min(100, n_simulations)

for i in range(n_paths_to_show):
    fig.add_trace(
        go.Scatter(x=days, y=portfolio_scenarios[i], 
                  mode='lines', line=dict(width=0.5, color='lightblue'),
                  showlegend=False),
        row=1, col=1
    )

# Add percentile bands
percentiles = [5, 25, 50, 75, 95]
colors = ['red', 'orange', 'green', 'orange', 'red']
for i, p in enumerate(percentiles):
    values = np.percentile(portfolio_scenarios, p, axis=0)
    fig.add_trace(
        go.Scatter(x=days, y=values, mode='lines',
                  name=f'{p}th percentile', line=dict(color=colors[i], width=2)),
        row=1, col=1
    )

# 2. Final value distribution
final_values = portfolio_scenarios[:, -1]
fig.add_trace(
    go.Histogram(x=final_values, nbinsx=50, name='Final Values',
                opacity=0.7, marker_color='skyblue'),
    row=1, col=2
)

# Add VaR lines
fig.add_vline(x=scenario_stats['var_95'], line_dash="dash", 
              line_color="red", annotation_text="95% VaR",
              row=1, col=2)
fig.add_vline(x=scenario_stats['var_99'], line_dash="dash", 
              line_color="darkred", annotation_text="99% VaR",
              row=1, col=2)

# 3. Risk metrics comparison
risk_metrics_values = [
    initial_portfolio_value - scenario_stats['var_95'],
    initial_portfolio_value - scenario_stats['var_99'],
    initial_portfolio_value - scenario_stats['cvar_95'],
    initial_portfolio_value - scenario_stats['cvar_99']
]
risk_labels = ['VaR 95%', 'VaR 99%', 'CVaR 95%', 'CVaR 99%']

fig.add_trace(
    go.Bar(x=risk_labels, y=risk_metrics_values,
           marker_color=['lightcoral', 'red', 'orange', 'darkred'],
           name='Risk Metrics'),
    row=2, col=1
)

# 4. Percentile analysis
percentile_range = np.arange(1, 100)
percentile_values = [np.percentile(final_values, p) for p in percentile_range]

fig.add_trace(
    go.Scatter(x=percentile_range, y=percentile_values,
               mode='lines', name='Percentiles', line=dict(color='purple')),
    row=2, col=2
)

fig.update_layout(
    title=f'🎲 Monte Carlo Risk Analysis ({n_simulations:,} simulations, {time_horizon_days} days)',
    height=800,
    showlegend=True
)

# Update axis labels
fig.update_xaxes(title_text="Days", row=1, col=1)
fig.update_yaxes(title_text="Portfolio Value ($)", row=1, col=1)
fig.update_xaxes(title_text="Portfolio Value ($)", row=1, col=2)
fig.update_yaxes(title_text="Frequency", row=1, col=2)
fig.update_yaxes(title_text="Risk Amount ($)", row=2, col=1)
fig.update_xaxes(title_text="Percentile", row=2, col=2)
fig.update_yaxes(title_text="Portfolio Value ($)", row=2, col=2)

fig.show()
print("📊 Monte Carlo visualization complete")

## 4. Stress Testing

Analyze portfolio performance under extreme market conditions.

In [ ]:
# Stress Testing Scenarios
class StressTestingFramework:
    def __init__(self):
        self.scenarios = {}
    
    def add_scenario(self, name, description, shocks):
        """
        Add a stress testing scenario
        
        shocks: dict with asset names as keys and shock multipliers as values
        """
        self.scenarios[name] = {
            'description': description,
            'shocks': shocks
        }
    
    def run_stress_test(self, current_portfolio_value, weights, asset_names):
        """
        Run stress tests on portfolio
        """
        results = {}
        
        for scenario_name, scenario in self.scenarios.items():
            portfolio_change = 0
            
            for i, asset in enumerate(asset_names):
                if asset in scenario['shocks']:
                    asset_value = current_portfolio_value * weights[i]
                    shock = scenario['shocks'][asset]
                    portfolio_change += asset_value * shock
            
            new_portfolio_value = current_portfolio_value + portfolio_change
            loss_amount = current_portfolio_value - new_portfolio_value
            loss_percentage = (loss_amount / current_portfolio_value) * 100
            
            results[scenario_name] = {
                'description': scenario['description'],
                'portfolio_value': new_portfolio_value,
                'loss_amount': loss_amount,
                'loss_percentage': loss_percentage
            }
        
        return results

# Define stress testing scenarios
stress_tester = StressTestingFramework()

# Historical crisis scenarios
stress_tester.add_scenario(
    'Market_Crash_2008',
    '2008-style financial crisis with severe equity and credit stress',
    {'Stock': -0.40, 'Bond': -0.10, 'Commodity': -0.35}
)

stress_tester.add_scenario(
    'COVID_Pandemic',
    'COVID-19 pandemic scenario with flight to quality',
    {'Stock': -0.30, 'Bond': 0.05, 'Commodity': -0.25}
)

stress_tester.add_scenario(
    'Interest_Rate_Shock',
    'Sudden 200bp interest rate increase',
    {'Stock': -0.15, 'Bond': -0.20, 'Commodity': -0.10}
)

stress_tester.add_scenario(
    'Inflation_Shock',
    'Unexpected inflation surge',
    {'Stock': -0.20, 'Bond': -0.25, 'Commodity': 0.15}
)

stress_tester.add_scenario(
    'Geopolitical_Crisis',
    'Major geopolitical event affecting global markets',
    {'Stock': -0.25, 'Bond': 0.03, 'Commodity': 0.20}
)

# Run stress tests
print("⚡ Running stress testing scenarios...")

stress_results = stress_tester.run_stress_test(
    current_portfolio_value=initial_portfolio_value,
    weights=weights,
    asset_names=list(returns_data.columns)
)

print("\n💥 STRESS TESTING RESULTS")
print("=" * 70)

for scenario, results in stress_results.items():
    print(f"\n📊 {scenario}:")
    print(f"   Description: {results['description']}")
    print(f"   Portfolio Value: ${results['portfolio_value']:,.2f}")
    print(f"   Loss Amount: ${results['loss_amount']:,.2f}")
    print(f"   Loss Percentage: {results['loss_percentage']:.2f}%")

print("\n✅ Stress testing complete")

In [ ]:
# Visualize stress testing results
scenario_names = list(stress_results.keys())
loss_amounts = [stress_results[s]['loss_amount'] for s in scenario_names]
loss_percentages = [stress_results[s]['loss_percentage'] for s in scenario_names]

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=['Stress Test Losses (Dollar Amount)', 'Stress Test Losses (Percentage)'],
    specs=[[{"type": "bar"}, {"type": "bar"}]]
)

# Dollar losses
fig.add_trace(
    go.Bar(x=scenario_names, y=loss_amounts,
           marker_color='red', opacity=0.7,
           name='Loss Amount ($)',
           text=[f'${x:,.0f}' for x in loss_amounts],
           textposition='auto'),
    row=1, col=1
)

# Percentage losses
fig.add_trace(
    go.Bar(x=scenario_names, y=loss_percentages,
           marker_color='darkred', opacity=0.7,
           name='Loss Percentage (%)',
           text=[f'{x:.1f}%' for x in loss_percentages],
           textposition='auto'),
    row=1, col=2
)

fig.update_layout(
    title='💥 Stress Testing Results',
    height=500,
    showlegend=False
)

fig.update_xaxes(tickangle=45)
fig.update_yaxes(title_text="Loss Amount ($)", row=1, col=1)
fig.update_yaxes(title_text="Loss Percentage (%)", row=1, col=2)

fig.show()

# Identify worst-case scenario
worst_scenario = max(stress_results.keys(), key=lambda x: stress_results[x]['loss_percentage'])
worst_loss = stress_results[worst_scenario]['loss_percentage']

print(f"🚨 Worst-case scenario: {worst_scenario} ({worst_loss:.2f}% loss)")
print("📊 Stress testing visualization complete")

## 5. Risk Summary and Recommendations

Comprehensive summary of risk analysis findings and actionable recommendations.

In [ ]:
# Generate comprehensive risk summary
print("🎯 COMPREHENSIVE RISK ANALYSIS SUMMARY")
print("=" * 60)

print(f"\n📊 PORTFOLIO OVERVIEW:")
print(f"   Initial Value: ${initial_portfolio_value:,}")
print(f"   Assets: {', '.join(returns_data.columns)}")
print(f"   Allocation: {', '.join([f'{w:.1%}' for w in weights])}")
print(f"   Analysis Period: {returns_data.index[0].strftime('%Y-%m-%d')} to {returns_data.index[-1].strftime('%Y-%m-%d')}")

print(f"\n⚡ RISK METRICS ({var_confidence*100}% confidence):")
print(f"   Monte Carlo VaR (30-day): ${initial_portfolio_value - scenario_stats['var_95']:,.2f}")
print(f"   Monte Carlo CVaR (30-day): ${initial_portfolio_value - scenario_stats['cvar_95']:,.2f}")
print(f"   Expected Portfolio Value: ${scenario_stats['mean']:,.2f}")
print(f"   Volatility Range: ${scenario_stats['min']:,.2f} - ${scenario_stats['max']:,.2f}")

print(f"\n💥 STRESS TESTING INSIGHTS:")
worst_scenario_details = stress_results[worst_scenario]
print(f"   Worst-case scenario: {worst_scenario}")
print(f"   Maximum potential loss: ${worst_scenario_details['loss_amount']:,.2f} ({worst_scenario_details['loss_percentage']:.2f}%)")
print(f"   Average stress loss: {np.mean(loss_percentages):.2f}%")

print(f"\n📈 INDIVIDUAL ASSET RISKS:")
for asset in returns_data.columns:
    vol = risk_df.loc[asset, 'Volatility']
    sharpe = risk_df.loc[asset, 'Sharpe_Ratio']
    var_hist = risk_df.loc[asset, 'Historical_VaR']
    print(f"   {asset}: Vol={vol:.1f}%, Sharpe={sharpe:.3f}, VaR={var_hist:.4f}")

print(f"\n💡 RISK MANAGEMENT RECOMMENDATIONS:")
print("   1. 🛡️  Consider reducing allocation to highest volatility assets")
print("   2. 📊  Implement regular VaR monitoring and backtesting")
print("   3. 🎯  Establish stop-loss levels based on stress test results")
print("   4. 🔄  Rebalance portfolio quarterly to maintain target allocation")
print("   5. 📈  Consider hedging strategies for tail risk protection")
print("   6. 🔍  Monitor correlation changes during market stress")

print(f"\n📋 RISK LIMITS SUGGESTIONS:")
print(f"   Daily VaR Limit: ${(initial_portfolio_value - scenario_stats['var_95'])/30:.0f} (1-day equivalent)")
print(f"   Monthly VaR Limit: ${initial_portfolio_value - scenario_stats['var_95']:,.0f}")
print(f"   Stress Test Threshold: {worst_loss * 0.8:.1f}% (80% of worst case)")
print(f"   Maximum Single Asset Weight: 40% (diversification)")

print(f"\n✅ Risk analysis complete! Regular monitoring and updates recommended.")